In [1]:
import numpy as np
from fractalai.policy import GreedyPolicy
from fractalai.model import RandomDiscreteModel
from fractalai.fractalai import FractalAI
from fractalai.environment import AtariEnvironment
from fractalai.monitor import AtariMonitorPolicy

In [2]:
# Environment Params
n_fixed_steps=4

name = "MsPacman-v0"
clone_seeds = True
max_steps = 1e6
skip_frames = 50
render=True
# FAI PARAMS
max_samples=15
max_states=15
time_horizon=10

In [3]:

env =  AtariEnvironment(name=name, clone_seeds=clone_seeds)
model = RandomDiscreteModel(env.env.action_space.n)
greedy = GreedyPolicy(env=env, model=model)

fractal = FractalAI(policy=greedy, max_samples=max_samples, max_states=max_states,
                    time_horizon=time_horizon, n_fixed_steps=n_fixed_steps)

In [4]:
monitor = AtariMonitorPolicy(policy=fractal, directory="videos", force=True)

## Record a video

This will make everything a bit slowe, but you will b

In [5]:
monitor.record_video(skip_frames=120)

In [ ]:
# With no video
# fractal.evaluate(render=render, max_steps=max_steps, skip_frames=skip_frames)